In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from depsurf import DepKind, DiffResult, VersionGroup
from utils import save_pkl, OUTPUT_PATH

results: DiffResult = {}

for group in [VersionGroup.ARCH, VersionGroup.FLAVOR]:
    result = group.diff_pairs(
        kinds=[
            DepKind.CONFIG,
            DepKind.FUNC,
            DepKind.STRUCT,
            DepKind.TRACEPOINT,
            DepKind.LSM,
            DepKind.SYSCALL,
        ],
        result_path=OUTPUT_PATH / "config" / group.name,
    )
    results.update(result)

save_pkl(results, "config")

[        utils.py:160] INFO: NumExpr defaulting to 8 threads.
[version_group.py:131] INFO: Diffing Arch
[version_group.py:137] INFO: Comparing x86_arm64 to /Users/szhong/Downloads/bpf-study/output/config/Arch/x86_arm64
[          btf.py:21 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/data/btf_norm/5.4.0-26-generic-amd64.pkl
[          btf.py:21 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/data/btf_norm/5.4.0-26-generic-arm64.pkl
[       symtab.py:70 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/data/symtab/5.4.0-26-generic-amd64.jsonl
[       symtab.py:70 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/data/symtab/5.4.0-26-generic-arm64.jsonl
[version_group.py:137] INFO: Comparing x86_arm32 to /Users/szhong/Downloads/bpf-study/output/config/Arch/x86_arm32
[          btf.py:21 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/data/btf_norm/5.4.0-26-generic-armhf.pkl
[       symtab.py:70 ] INFO: Loading symtab from /Users

In [1]:
import pandas as pd

from depsurf import DepKind, DiffResult, IssueEnum
from utils import load_pkl, GRAY_DASH, save_latex

data: DiffResult = load_pkl("config")

ISSUES = {
    IssueEnum.ADD: "$+$",
    IssueEnum.REMOVE: "$-$",
    IssueEnum.CHANGE: "$\Delta$",
    IssueEnum.NEW: "#",
}

results = {}
for (versions, pair), d in data.items():
    if pair.v2.flavor == "oracle":
        continue
    result = {}
    for (kind, s), v in d.items():
        if kind == DepKind.LSM:
            kind = "LSM"
        if s not in ISSUES.keys():
            continue
        s = ISSUES[s]
        if v > 1000:
            v = f"{v/1000:.1f}k"
        result[(kind, s)] = v
    results[(versions, versions.version_to_str(pair.v2))] = result


df = pd.DataFrame(results)
df = df.loc[(df != 0).any(axis=1) | (df.index.get_level_values(1) == "$\Delta$")]


df_latex = df.map(lambda x: GRAY_DASH if x == 0 else x)
latex = df_latex.to_latex(multicolumn_format="c|", column_format="ll|rrrr|rrrr")

save_latex(latex, "cfg")
df

[          pkl.py:18 ] INFO: Loding config from /Users/szhong/Downloads/bpf-study/output/config.pkl
[        latex.py:44 ] INFO: Saved cfg to /Users/szhong/Downloads/bpf-study/paper/tabs/cfg.tex


Arch                      Flavor                     
                     arm64  arm32    ppc   s390    Lat    AWS  Azure    GCP
Config     #          9.6k   9.6k   8.1k   3.0k   8.8k   6.4k   5.3k   8.6k
           $+$        1.9k   2.2k    657    206      9      8     11      0
           $-$        1.1k   1.4k   1.3k   5.9k      6   2.4k   3.5k    190
           $\Delta$     41     85     51    533      9    252    871    249
Function   #         49.4k  48.8k  42.8k  33.2k  48.0k  46.5k  45.5k  48.1k
           $+$        9.2k  12.6k   5.4k   3.9k     57    328    992    450
           $-$        7.9k  11.8k  10.6k  18.8k     41   1.8k   3.5k    319
           $\Delta$    120    106    137     78      0      2     10      1
Struct     #          9.1k   8.6k   7.4k   6.1k   8.4k   8.0k   7.8k   8.4k
           $+$        1.7k   2.0k    570    694      4     83    257     68
           $-$        1.0k   1.9k   1.6k   3.1k      1    483    833    123
           $\Delta$     81    154    116    115      5     19     28     14
Tracepoint #           685    690    648    559    752    747    739    752
           $+$          45     70     25     61      0      4     26      0
           $-$         112    132    129    254      0      9     39      0
           $\Delta$     17     17     18     21      3      0      0      0
LSM        #           189    189    189    189    189    189    189    189
           $\Delta$      0      0      0      0      0      0      0      0
Syscall    #           291    378    347    341    333    333    333    333
           $+$           2     74     23     22      0      0      0      0
           $-$          44     29      9     14      0      0      0      0
           $\Delta$      0      0      0      0      0      0      0      0